The lecture concers the realdata set analyze on three distinct seed types. For some of ML algorithms it is a good idea to scale the data. It even has it's own name: Curse of dimensionality(Przekleństwo wymiarowości).

- The curse of dimensionality refers to various phenomena that arise when analyzing and organizing data in high-dimensional spaces that do not occur in low-dimensional settings such as the three-dimensional physical space of everyday experience. The expression was coined by Richard E. Bellman when considering problems in dynamic programming.

In [1]:
import findspark
findspark.init('/home/ubuntu/Spark/spark-3.3.0-bin-hadoop3')

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Clustering_CodeAlong').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/15 10:41:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/09/15 10:41:26 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/09/15 10:41:26 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [3]:
dataset = spark.read.csv('seeds_dataset.csv', header=True, inferSchema=True)

In [4]:
dataset.printSchema()

root
 |-- area: double (nullable = true)
 |-- perimeter: double (nullable = true)
 |-- compactness: double (nullable = true)
 |-- length_of_kernel: double (nullable = true)
 |-- width_of_kernel: double (nullable = true)
 |-- asymmetry_coefficient: double (nullable = true)
 |-- length_of_groove: double (nullable = true)



In [5]:
dataset.head(1)

[Row(area=15.26, perimeter=14.84, compactness=0.871, length_of_kernel=5.763, width_of_kernel=3.312, asymmetry_coefficient=2.221, length_of_groove=5.22)]

In [6]:
from pyspark.ml.clustering import KMeans

In [7]:
from pyspark.ml.feature import VectorAssembler

In [8]:
dataset.columns

['area',
 'perimeter',
 'compactness',
 'length_of_kernel',
 'width_of_kernel',
 'asymmetry_coefficient',
 'length_of_groove']

In [9]:
assembler = VectorAssembler(inputCols=dataset.columns, outputCol='features')

In [10]:
final_data = assembler.transform(dataset)

In [11]:
final_data.printSchema()

root
 |-- area: double (nullable = true)
 |-- perimeter: double (nullable = true)
 |-- compactness: double (nullable = true)
 |-- length_of_kernel: double (nullable = true)
 |-- width_of_kernel: double (nullable = true)
 |-- asymmetry_coefficient: double (nullable = true)
 |-- length_of_groove: double (nullable = true)
 |-- features: vector (nullable = true)



In [12]:
from pyspark.ml.feature import StandardScaler

In [13]:
scaler = StandardScaler(inputCol='features', outputCol='scaledFeatures' )

In [14]:
scaler_model = scaler.fit(final_data)

In [15]:
final_data = scaler_model.transform(final_data)

In [16]:
final_data.head(1)

[Row(area=15.26, perimeter=14.84, compactness=0.871, length_of_kernel=5.763, width_of_kernel=3.312, asymmetry_coefficient=2.221, length_of_groove=5.22, features=DenseVector([15.26, 14.84, 0.871, 5.763, 3.312, 2.221, 5.22]), scaledFeatures=DenseVector([5.2445, 11.3633, 36.8608, 13.0072, 8.7685, 1.4772, 10.621]))]

In [17]:
kmeans = KMeans(featuresCol='scaledFeatures', k=3)

In [18]:
model = kmeans.fit(final_data)

In [20]:
print('WSSSE')
print(model.summary.trainingCost)

WSSSE
429.00761965459367


In [21]:
centers = model.clusterCenters()

In [22]:
print(centers)

[array([ 4.90455443, 10.919579  , 37.26051182, 12.3885095 ,  8.57467662,
        1.81659031, 10.38074771]), array([ 4.06133795, 10.13721767, 35.82681204, 11.81771972,  7.5087187 ,
        3.25852121, 10.4215732 ]), array([ 6.35645488, 12.40730852, 37.41990178, 13.93860446,  9.7892399 ,
        2.41585013, 12.29286107])]


In [24]:
model.transform(final_data).select('prediction').show()

+----------+
|prediction|
+----------+
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         2|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         1|
+----------+
only showing top 20 rows

